In [ ]:
from functions.functions_Operation import *


############################
#### Single Node Operation #
############################

Zones="FR"
year=2013

Selected_TECHNOLOGIES={'Thermal', 'OldNuke', 'NewNuke', 'HydroRiver', 'HydroReservoir',
       'WindOnShore', 'WindOffShore', 'Solar', 'Curtailement'}

#### reading CSV files
areaConsumption = pd.read_csv('CSV/input/areaConsumption'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
availabilityFactor = pd.read_csv('CSV/input/availabilityFactor'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
TechParameters = pd.read_csv('CSV/input/Gestion-Simple_TECHNOLOGIES.csv',sep=';',decimal=',',skiprows=0)

#### Selection of subset
availabilityFactor=availabilityFactor[ availabilityFactor.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
TechParameters=TechParameters[TechParameters.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]

availabilityFactor.head()

availabilityFactor.pivot(index="TIMESTAMP",columns='TECHNOLOGIES', values='availabilityFactor').\
    iloc[1:240].plot(y="WindOnShore",kind="line").set_ylim(0,1)

model = GetElectricSystemModel_GestionSingleNode(areaConsumption,availabilityFactor,TechParameters)
opt = SolverFactory('mosek')
results=opt.solve(model)


#### obtain optimized variables in panda form
Variables=getVariables_panda(model)
Variables.keys()

Variables["energy"].pivot(index="TIMESTAMP",columns='TECHNOLOGIES', values='energy').\
    iloc[1:24].plot.area()

#### obtain lagrange multipliers in panda form
Constraints= getConstraintsDual_panda(model)
Constraints.keys()
Constraints["energyCtr"].assign(Prix=lambda x: x.energyCtr *10**6).head()
Constraints["energyCtr"].assign(Prix=lambda x: x.energyCtr *10**6).iloc[1:8760].\
    plot(x="TIMESTAMP",y="Prix",kind="line").set_ylim(0,200)
Constraints["energyCtr"].head()
